# 記法について

* Array#each : Arrayクラスのインスタンスメソッドeach
* Thread.fork : Threadクラスのクラスメソッドfork
* Math.#sqrt : Mathモジュールのモジュール関数sqrt

# 識別子

* クラス名や変数名などを構成するもの。全ての文法の基盤
* 予約語は識別子として利用できない
    * nil, true, false, not, or, and, do, then, yield, rescure, unsure,
    * class, module, def, undef, define?, alias, super, self, return,
    * while, until, for, in, break, next, case, when, if, unless, else, elsif
    * BEGIN, END, begin, end, redo, retry, 
    * `__LINE__, __FILE__, __ENCODING__`

# 変数
 * rubyは変数の初期化時に変数の型を指定しない「動的型言語」

In [1]:
a = 1
p a
p a.class
a = "foo"
p a.class

1
Fixnum
String


String

## ローカル変数

* ローカル変数の識別子
    * アンダースコアと英数字
* 先頭に数字は利用できない
* アンダースコア以外の記号は使えない
    * ダメな例
        * hoge-1
        * initialized?
        * 1_to_10
* スコープ
    * その代入を含むブロックまで
    * メソッドの終わりまで
* 初期化されていないときの参照
    1. 代入文が実行されなかったときはnil
    1. 代入文が無い場合は例外(NameError)

In [5]:
### 代入分が実行されなかったときはnil
abb =
p abb

### 代入文が無い場合は例外(NameError)
p bbb

nil


NameError: undefined local variable or method `bbb' for main:Object

## インスタンス変数

* 命名規則
    * 先頭
        * `@`
    * 構成文字
        * 英数字又はアンダースコア
* スコープ
    * そのインスタンス内
* 初期化されていない時の参照
    * nil

In [12]:
### インスタンス変数の初期化
@a = 5
p @a

### 初期化されていないインスタンス変数を参照
p @b



5
nil


In [19]:
### @の直後に数字は設定できない(SyntaxError)
@1 = 10
p @1

SyntaxError: (eval):3: `@1' is not allowed as an instance variable name


## クラス変数

* 命名規則
    * 先頭
        * `@@`
    * 構成文字
        * 英数字又はアンダースコア
* スコープ
    * そのクラスの全インスタンス
* 初期化されていない時の参照
    * 例外発生(NameError)

In [16]:
### クラス変数の初期化
@@a = 5
p @@a

### 初期化されていないクラス変数を参照
p @@b

(pry):66: warning: class variable access from toplevel
(pry):67: warning: class variable access from toplevel


5


(pry):70: warning: class variable access from toplevel


NameError: uninitialized class variable @@b in Object

In [20]:
### @@の直後に数字は設定できない
@@1 = 10
p @@1

SyntaxError: (eval):3: `@@1' is not allowed as a class variable name


## グローバル変数

* 命名規則
    * 先頭
        * `$`
    * 構成文字
        * 英数字又はアンダースコア
* スコープ
    * どこからでも参照可能
* 初期化されていない時の参照
    * nil

In [22]:
### クラス変数の初期化
$a = 5
p $a

### 初期化されていないグローバル変数を参照
p $b

5
nil


In [21]:
### $の直後に数字は設定できない
$1 = 10
p $1

SyntaxError: (eval):3: Can't set variable $1


## 定数

* Rubyは定数の値を変更できるという意味では変数とも言える
* 命名規則
    * 先頭
        * 英数大文字
    * 構成文字
        * 英数字又はアンダースコア
* スコープ
    * 定数が定義されたクラス・モジュール
    * そのクラス・モジュール内で定義されたクラス・モジュール
    * そのクラス・モジュールをインクルードしているモジュール
    * クラス名やモジュール名で修飾すれば外部からアクセス可能
* 初期化されていない時の参照
    * 例外発生(NameError)

In [25]:
### 変数名の先頭を大文字にすると定数になる
AAA = 1
p AAA

### 警告は出るが再代入はできる
AAA = 2
p AAA

### 初期化されていない時の参照は例外が発生
p BBB

(pry):108: warning: already initialized constant AAA
(pry):103: warning: previous definition of AAA was here


1


(pry):112: warning: already initialized constant AAA
(pry):108: warning: previous definition of AAA was here


2


NameError: uninitialized constant BBB

# リテラル

* プログラムにそのまま記述する値のこと
* 変数と対になるもの

# 数値リテラル

* 例
    * +12, -12, 0.1, 3.0e2(300.0), 3.0e-2(0.03)
* 基数指示子で進数を表現できる
    * 2進数(0b)
    * 8進数(0o or 0)
    * 10進数(0d)
    * 16進数(0x)
* アンダースコアが使える
    * 100_000_000 = 100000000

In [41]:
### 2進数
p "2進数"
p 0b10
p 0b100

### 8進数
p "8進数"
p 0o10
p 010
p 012

### 10進数
p "10進数"
p 0d10
p 10
p 0d123

### 16進数
p "16進数"
p 0x10
p 0x9
p 0xA
p 0xF

### アンダースコア
p "アンダースコア 100_000_000"
p 100_000_000

"2進数"
2
4
"8進数"
8
8
10
"10進数"
10
10
123
"16進数"
16
9
10
15
"アンダースコア 100_000_000"
100000000


100000000

## 有理数のリテラル

* 例
    * 42/10r
    * 3.14r

## 複素数のリテラル

* 例
    * 1+3i

In [43]:
### 有理数のリテラル
p 42/10r
p 3.14r

### 複素数のリテラル
p 1+3i

(21/5)
(157/50)
(1+3i)


(1+3i)

# 数値演算 (比較)

* 真の場合true、偽の場合falseが返る
* 例
    * 1 == 1
    * 1 != 2
    * 1 < 2
    * 2 >= 2
* 以下の演算子はオーバーライドできない
    * !=
    * !~

# UFO演算子

* 比較の結果を数値で返す
    * 例
        * A <=> B
        * AがBより大きい  : 1
        * AとBが同じ     : 0
        * AがBより小さい  : -1

# 自己代入演算子

* Rubyには++, -- のようなインクリメント、デクリメント演算子は存在しない
    * 代わりに自己代入演算子を使う
    * 例
        * a += 1
        * a -= 1
        * a *= 2
        * a **=2
* 最初に左のオペランドを評価する
    * <font color="red">変数に値が代入されていない場合参照でNoMethodErrorが発生する</font>
* オーバーライドできない

In [6]:
### 比較演算子
### 正しいとtrue、違っているとfalseが返る
puts "比較演算子".force_encoding("utf-8")
p 5 == 5
p 5 < 4

### UFO演算子
puts "UFO演算子".force_encoding("utf-8")
p 5 <=> 5
p 5 <=> 4
p 5 <=> 6

### 自己代入演算子
puts "自己代入演算子".force_encoding("utf-8")
a = 5
b = 5
p a *= 3
p b **=3
### cは初期化されていないのでNoMethodErro
#c += 1

比較演算子
true
false
UFO演算子
0
1
-1
自己代入演算子
15
125


125

# 継承クラスの参照

In [57]:
p 1.class # Fixnum
p 1.class.superclass # Integer
p 1.class.superclass.superclass # Numeric
p 1.class.superclass.superclass.superclass # Object
p 1.class.superclass.superclass.superclass.superclass # BasicObject
p 1.class.superclass.superclass.superclass.superclass.superclass # nil

Fixnum
Integer
Numeric
Object
BasicObject
nil


# 加算は実際は「+メソッド」を実行している

In [58]:
p 1.+(3)

4


4

# 再定義(オーバーライド)できない演算子

* スコープ演算子
    * ::
* 代入演算子
    * =
* 条件演算子
    * ?
* 範囲演算子
    * .., ...
* 論理演算子
    * &&
    * and
    * ||
    * or
    * not

# 論理値

* true と false の2つの値が存在する
    * TrueClass と FalseClassのインスタンス
* falseとnil以外のオブジェクトはすべて真とみなされる

In [62]:
p true.class
p false.class

TrueClass
FalseClass


FalseClass

# 論理演算子

* 論理積(かつ)
    * &&
* 論理和(または)
    * ||
* 否定(でない)
    * !

In [70]:
if true && true;    p "ok01"; end #=> "ok01"
if true && false;   p "ok02"; end #=>
if false && true;   p "ok03"; end #=>
if false && false;  p "ok04"; end #=>

if true || true;    p "ok05"; end #=> "ok05"
if true || false;   p "ok06"; end #=> "ok06"
if false || true;   p "ok07"; end #=> "ok07"
if false || false;  p "ok08"; end #=>

if !true;           p "ok09"; end #=>

if true && !false;  p "ok10"; end #=> "ok10"

if true && nil;     p "ok11"; end #=>

if true && !nil;     p "ok12"; end #=> "ok12"

p "左辺で結果が確定する場合は右辺を評価しない"
if true || false && false;    p "ok13"; end #=> "ok13" ### 最初のtrueで確定している
if true && true  || false;    p "ok14"; end #=> "ok14" ### 真ん中のtrueで確定している
if true && true  && false;    p "ok15"; end #=>        ### 最後のfalseで確定している

### 最初で確定
p nil  && false     #=> nil
p false && nil       #=> false

### 最後で確定
p nil || false     #=> false
p false || nil       #=> nil

p "論理演算子を利用した式の評価"
p a = 1     && 2     #=> 2     ### trueが確定する右辺でその値の2が返る
p b = nil   && 3     #=> nil   ### 左辺でnilが確定するので右辺の3は評価されない
p c = 4     && nil   #=> nil
p d = 5     && false #=> false
p e = false && 6     #=> false

p f = 7     || 8     #=> 7     ### 左辺でtrueが確定するのでその値の7が返る
p g = nil   || 9     #=> 9     ### trueが確定する右辺でその値の9が返る

"ok01"
"ok05"
"ok06"
"ok07"
"ok10"
"ok12"
"左辺で結果が確定する場合は右辺を評価しない"
"ok13"
"ok14"
nil
false
false
nil
"論理演算子を利用した式の評価"
2
nil
nil
false
false
7
9


9

## 論理演算子の自己代入

* 最後に評価したオペランドの値を返すことを利用した変数のデフォルト値の設定

In [81]:
#p aaa bbb           #=> NameError
p aaa = aaa || 1    #=> 1   ### aが初期化されていないときに1を代入
p bbb ||= 2           #=> 2   ### bが初期化されていないときに2を代入

ccc = 3
p ccc &&= 4         #=> 4   ### cが初期化(値は3)されているので4を代入
p ddd &&= 5         #=> nil ### dが初期化されていないのでnilが入る

1
2
4
nil


## 特徴的な論理演算子

* 例
  * and
  * or
  * not
* &&や||や!との違い
  * 自己代入できない
  * 代入演算子よりも演算子の優先度が低い


In [85]:
### 自己代入できない
#p a or= 1        #=> SyntaxEror

### 代入演算子よりも演算子の優先度が低い
p a = 1 &&  2      #=> 2   ### p (1 and 2)   と同じ。論理積の結果がpに渡される
p b = (1 and 2)    #=> 2
p c = 1 and 2      #=> 1   ### p (1) and 2   と同じ。
p d = 3 and nil    #=> 3   ### p (3) and nil と同じ。

2
2
1
3


# 条件分岐

```
if <条件式> then
〜
end
```
<条件式>が真のとき〜の処理がされる

unlessもある

In [89]:
### 条件が成立したときに値を代入するという書き方ができる
a = if true
      1
    end
b = 2 if true
cccc = 3 if false  ### 3は代入されない (代入文はあるが実行されないが、判定の結果にかかわらず変数自体は確保されている)
p a
p b
p cccc ### Errorにはならずnilが出力される

### elsifとelse
a = if false then
      1
    elsif false
      2
    else
      3
    end
p a   # 3

1
2
nil
3


3

# 三項演算子 (条件演算子)

```
条件式 ? 式1 : 式2
```
条件式を評価し、成立するときは式1、しない場合は式2を評価して返す

In [92]:
a = true ? 1 : 2
b = false ? 1 : 2
p a
p b

1
2


2

# 擬似変数

* 記述しても新たに値は生成されず、唯一のインスタンスが参照される
    * true
        * TrueClassのインスタンス
    * false
        * FalseClassのインスタンス
    * nil
        * NilClassのインスタンス
            * 何もない事を表現する
            * 他のプログラミング言語ではnullに相当
            * インスタンス変数の初期値に利用
    * self
        * 現在のオブジェクト
        
    * `__FILE__`
        * 現在実行しているプログラムのファイル名
    * `__LINE__`
        * 現在実行しているプログラムの行番号    
    * `__ENCODING__`
        * 現在のソースファイルのスクリプトエンコーディング

# 文字のリテラル

* 「R」を表すString
     * ?R
* Ctrl+vを表すString
    * ?\C-v

In [93]:
p ?R
p ?\C-v

"R"
"\u0016"


"\u0016"

# 文字列

* リテラル書式のバリエーションが豊富 (基本的な書式、ヒアドキュメント、パーセント記法)

## 基本的な書式

* ダブルクオート「"」
    * 式展開できる
* シングルクオート「'」
    * 式展開できない

In [97]:
p "ダブルクオート"
a = "xx"
p a.class     #=> String
b = "xx" "yy"
p b           #=> "xxyy"
p "#{b}zz"    #=> "xxyyzz"   ### b.to_s のようにto_sメソッドが呼び出されている

p "シングルクオート"
e = 'xx' 'yy'
p e           #=> "xxyy"
p '#{e}zz'    #=> "\#{e}zz"

p "文字列から数値に変換"
p "123".to_i
p "456hoge789".to_i  #=> 456   ### hogeの前までが変換対象
p "hoge".to_i        #=> 0     ### 無効の場合は0が返る
p "1.23".to_i        #=> 1     ### ピリオドの前までが変換対象
p "1.23".to_f        #=> 1.23
p "4.56.789".to_f    #=> 4.56  ### 2番目のピリオドの前までが返還対象
p "4.5hoge6".to_f    #=> 4.5

"ダブルクオート"
String
"xxyy"
"xxyyzz"
"シングルクオート"
"xxyy"
"\#{e}zz"
"文字列から数値に変換"
123
456
0
1
1.23
4.56
4.5


4.5

## バックスラッシュ記法


|記入|意味|
|---|---|
|\x|xそのもの|
|\n|改行|
|\s|空白|
|\b|バックスペース|
|\t|タブ|
|\v|垂直タブ|
|\r|キャリッジリターン|
|\a|ベル|
|\e|エスケープ|
|\nnn|8進数表記(nは0-7)|
|\xnn|16進数表記 (nは0-9,a-f)|
|\cx または \C-x|コントロール文字 (xはASCII文字)|
|\M-x|メタx|
|\M-\C-x|メタコントロールx|
|\unnnn|ユニコード文字(nは0-9,a-f,A-F)|
|\u{nnnn}|ユニコード文字列 スペースかタブ区切りで複数指定可能|

In [103]:
### "A"の8進数表記と16進数表記。表示したい文字コードの数値を指定する
p "\101"
p "\x41"

"A"
"A"


"A"

In [111]:
### p,puts,printの違い
p "\101"          #=> "A" 引数ごとに改行、inspectメソッド、そのまま出力
puts "\101"       #=> A　改行しない、to_sメソッド、適用した結果を出力
print "\101"     #=> A　引数ごとに改行、to_sメソッド、適用した結果を出力

"A"
A
A

## ヒアドキュメント

* 基本
    * 「<<」に続けて文字列の終端を表す任意の識別子を指定
* 変数に代入
* 階層が深い場合
    * 通常終端を表す識別子の前にスペースなどの文字を記述してはいけない
    * これを回避するには識別子の頭に「-」をつける
* 式展開
    * デフォルト、または「"」で囲むと式展開できる
    * 「'」で囲むとそのまま出力される

In [128]:
### 基本
p <<EOS   #=> "hoge\n"
hoge
EOS

### 変数に代入
foo = <<EOS
var
EOS
p foo    #=> "var\n"

### 識別子の頭に「-」をつけてスペースを入れられるようにする
p <<-EOS   #=> "fuga\n"
fuga
  EOS

### 変数展開
a = "var"
p <<EOS    #=> "var\n"
#{a}
EOS
p <<"EOS"  #=> "var\n"
#{a}
EOS

### 変数展開されない
p <<'EOS'  #=> "#{a}\n"
#{a}
EOS

"hoge\n"
"var\n"
"fuga\n"
"var\n"
"var\n"
"\#{a}\n"


"\#{a}\n"

## パーセント記法

* 文字列を囲む記号をプログラマが指定できる
    * 例えば、文字列の中でダブルクオートを使うときにエスケープしなくてよくなる
    * カッコを使う場合は終端は対応する閉じカッコを使う

In [130]:
### 文字列を囲む記号の例
p %&He said "Hello"&   #=> "He said \"Hello\""
p %@He said "Hello"@   #=> "He said \"Hello\""
p %+He said "Hello"+   #=> "He said \"Hello\""

### カッコを使う場合は終端は対応する閉じカッコを使う
# p %{He said "Hello"{   #=> SyntacError
p %{He said "Hello"}   #=> "He said \"Hello\""
p %(He said "Hello")   #=> "He said \"Hello\""
p %[He said "Hello"]   #=> "He said \"Hello\""
p %<He said "Hello">   #=> "He said \"Hello\""

"He said \"Hello\""
"He said \"Hello\""
"He said \"Hello\""
"He said \"Hello\""
"He said \"Hello\""
"He said \"Hello\""
"He said \"Hello\""


"He said \"Hello\""

|記法|生成される文字列|
|---|---|
|%|ダブルクオート文字列|
|%Q|ダブルクオート文字列 (%だけと同じ)|
|%q|シングルクオート文字列|
|%s|シンボル。式展開無し|
|%W|要素がダブルクオート文字列となる配列。要素の区切りは空白文字列|
|%w|要素がシングルクオート文字列となる配列。要素の区切りは空白文字列|
|%l|要素がシンボルの配列。式展開する|
|%i |要素がシンボルの配列。式展開しない|
|%x|コマンド出力。バッククオートと同じ|
|%r|正規表現|


In [131]:
a = "Hello"
p  %{He said "#{a}"}   #=> "He said \"Hello\""
p %Q{He said "#{a}"}   #=> "He said \"Hello\""
p %q{He said "#{a}"}   #=> "He said \"\#{a}\""
p %s{foo}              #=> :foo

b = "pen"
p %W{I have a #{b} .}   #=> ["I", "have", "a", "pen", "."]
p %w{I have a #{b} .}   #=> ["I", "have", "a", "\#{b}", "."]
p %I{I have a #{b} .}   #=> [:I, :have, :a, :pen, :"."]
p %i{I have a #{b} .}   #=> [:I, :have, :a, :"\#{b}", :"."]

p %x{date}              #=> "2017年  1月 13日 金曜日 09:33:09 JST\n"
p %r(^http://)          #=> /^http:\/\//
p %q(^http://)          #=> "^http://"

"He said \"Hello\""
"He said \"Hello\""
"He said \"\#{a}\""
:foo
["I", "have", "a", "pen", "."]
["I", "have", "a", "\#{b}", "."]
[:I, :have, :a, :pen, :"."]
[:I, :have, :a, :"\#{b}", :"."]
"2017年 5月 6日 土曜日 11時56分49秒 JST\n"
/^http:\/\//
"^http://"


"^http://"

## 文字列演算

In [134]:
### +メソッドで文字連結
p "a" + "b"       #=> "ab"

### *メソッドで繰り返し出力
p "hoge" * 3      #=> "hogehogehoge"

### 左辺に数値を指定するとエラーになる
#p 3 * "hoge"      #=> TypeError

### << メソッドにより末尾に連結
p "hoge" << "fuga"  #=> "hogefuga"

### 異なるエンコード間で文字列操作を行うと例外が発生する (Ruby2.1まで？Ruby2.3.1はエラーにならない)
p "hoge".encoding                 #=> #<Encoding:UTF-8>
p "fuga".encode("SJIS").encoding  #=> #<Encoding:Windows-31J>
p "hoge" + "fuga".encode("SJIS")  #=> "hogefuga"  参考書的にはエラーになる

### 文字列は文字コードで大小を比較する
p "a" < "b"       #=> true
p "ab" < "ac"     #=> true
p "Ab" < "Ab"     #=> false
p "Ab" == "Ab"    #=> true
p "Aa" <=> "Ab"   #=> -1
p "Ab" <=> "Ab"   #=> 0
p "Ac" <=> "Ab"   #=> 1

### 文字数の確認
p "abcde".length      #=> 5
p "abcde".size        #=> 5
p "あいうえお".length  #=> 5
p "あいうえお".size    #=> 5

"ab"
"hogehogehoge"
"hogefuga"
#<Encoding:UTF-8>
#<Encoding:Windows-31J>
"hogefuga"
true
true
false
true
-1
0
1
5
5
5
5


5

## sprintfによる整形

* 帳票などで行数をそろえるときなどに利用
    * 第一引数 : フォーマット, 第二引数以降 : フォーマットしたい値

In [136]:
### 進数の指定
p sprintf("\%#b", 3)    #=> "0b11"  2進数
p sprintf("\%#o", 8)    #=> "010"   8進数
p sprintf("\%#x", 10)   #=> "0xa"  16進数
p sprintf("\%#X", 10)   #=> "0XA"  16進数 (大文字)

### 桁数の指定
p sprintf("%2d", 1)          #=> " 1"
p sprintf("%02d",1)          #=> "01"
p sprintf("%3d", 1)          #=> "  1"
p sprintf("%03d",1)          #=> "001"
p sprintf("%05.2f",123.4567) #=> "123.46"
p sprintf("%05.3f",123.4567) #=> "123.457"
p sprintf("%05.4f",123.4567) #=> "123.4567"

### sprintf関数はStringクラスの%演算と同じ結果を得られる
p "%02d" % 1  #=> "01"
p "%03d" % 1  #=> "001"

"0b11"
"010"
"0xa"
"0XA"
" 1"
"01"
"  1"
"001"
"123.46"
"123.457"
"123.4567"
"01"
"001"


"001"

# シンボル

* 多くのRuby処理系ではシンボルは内部では整数として扱われる
    * 文字列と比較して処理が速くなる
        * 単にラベルとして文字列を使う場合はシンボルの方が効率が良い
            * ハッシュのキーなどによく利用される
* (Ruby2.1以前)ガベージコレクションの対象外
    * キーの数が限られたケース以外では文字列を使う方が適切(Ruby2.1以前)
* 生成された値はSymbolクラスのインスタンスとなる
    * 「+」といった文字の並びを操作するメソッドが定義されていない
        * 動的にシンボルを生成する場合は文字列に変換してからシンボルに変換するという事が良く行われる

In [137]:
### シンボルは文字列の先頭にコロンをつける
p :"foo"       #=> :foo

### 文字列の囲み(ダブルクオート)は省略できる。省略するのが一般的
p :foo         #=> :foo

### パーセント記法でシンボルを生成
p %s(foo)      #=> :foo

### 動的にシンボルを生成する場合は文字列に変換してからシンボルに変換するという事が良く行われる
a = "foo"
p :"#{a*2}bar"             #=> :foofoobar
p "#{a*2}bar".to_sym       #=> :foofoobar
p "#{a*2}bar".to_sym.to_s  #=> "foofoobar"

:foo
:foo
:foo
:foofoobar
:foofoobar
"foofoobar"


"foofoobar"

## オブジェクトの同値性と同一性

* 同値性
    * 同じ値である。オブジェクトは異なる可能性がある
* 同一性
    * 同じオブジェクトである。(値が同じである必要は無いが、必然的に値は同じになるはず。。)
* シンボルについて
    * 文字の並びが同じであれば同一のオブジェクトを参照する
* 文字列について
    * 文字の並びが同じでも、指定するごとにあらたなStringオブジェクトが生成される
* オブジェクトID
    * リテラル(数値や文字列など)を指定
        * Rubyインタプリタがそのリテラル型に対応するクラスのインスタンスを生成
        * 生成されたインスタンスはすべてオブジェクトであり、一意のオブジェクトIDを持つ

In [139]:
### 文字列は別オブジェクトID、シンボルは同じオブジェクトID
p "foo".object_id     #=> (70353350030900などオブジェクトIDが表示される)
p "foo".object_id     #=> (70353350030760など上と違う値)
p :foo.object_id    #=> (2465848などオブジェクトIDが表示される)
p :foo.object_id    #=> (2465848など上と同じ値)

### nilやtrue、falseなども一つのオブジェクトを参照するためobject_idは変わらない
p nil.object_id       #=> 8
p nil.object_id       #=> 8
p true.object_id      #=> 20
p true.object_id      #=> 20
p false.object_id     #=> 0
p false.object_id     #=> 0

### オブジェクトの比較
p "foo".equal? "foo"  #=> false ### オブジェクトが同一かどうかを判定。同一でないのでfalse
p "foo" === "foo"     #=> false ### equal?と同じ。case式でよく使われる
p "foo".eql? "foo"    #=> true  ### 等価演算子で二つのオブジェクトが等しいかどうかを判定。等しいのでtrue
p "foo" == "foo"      #=> true  ### 基本的にeql?と同様だが、型の比較は行わない
p 1.to_f.eql? 1       #=> false ### 値は同じでも型が違うのでfalse
p 1.to_f == 1         #=> true  ### 型は違うが値が同じなのでtrue

70099478379920
70099478361260
3255708
3255708
8
8
20
20
0
0
false
true
true
true
false
true


true

# 変数代入の挙動の詳細

* 1 リテラルを指定すると対応するオブジェクトがメモリ上に生成される
* 2 変数を宣言すると変数にオブジェクトへの参照が与えられる
    * 変数を他の変数に代入すると両者は同じオブジェクトを指す
* 3 変数を参照すると参照先のオブジェクトが返される
* 4 変数に代入すると大部分は再代入になる
    * 片方の変数に代入しても、もう片方の変数に影響しない
        * 代入した方の変数には新たなオブジェクトへの参照が与えられる
        * 代入してない方の変数のオブジェクトへの参照はそのまま

In [143]:
a = "hoge"
b = a
p a.object_id  #=> (70169318563740など)
p b.object_id  #-> (70169318563740などaと同じ)
a = "fuga"     ### (aに別の値を代入)
p a            #=> "fuga"
p b            #=> "hoge"
p a.object_id  #=> (70353658141280など先ほどと違う)
p b.object_id  #=> (70169318563740など先ほどと同じまま)

### メソッドの引数も同様
### -> 実引数に指定された変数の参照が仮引数にコピーされる
p "メソッドの引数"
def func(y)       ### メソッドfuncを定義。yを仮引数と呼ぶ。
  p y.object_id   ### 1.仮引数yのオブジェクトIDを表示する
  y = "var"
  p y.object_id   ### 2.代入後のオブジェクトIDを表示
end
x = "foo"       ### 変数を宣言
p x.object_id   ### 変数xのオブジェクトIDを表示                            (70151085970820など)
func(x)         ### 1.仮引数yのオブジェクトIDを表示                        (70151085970820など上と同じ)
#                 ### -> 変数xのオブジェクトの参照が、仮引数yにコピーされている
#                 ### -> funcのスコープ内からスコープ外のメモリ領域を参照できている
#                 ### 2.続けてy = "var"が代入された後のyのオブジェクトIDを表示 (70181934504580など上と異なる)
#                 ### func(x)としたとき、変数xをメソッドfuncの「実引数」と呼ぶ。
#                 ### -> 二つ目のyのオブジェクトIDは再代入が行われているので値が異なる
#                 ### -> funcのスコープ内のメモリ領域に別のオブジェクトが生成され、そのオブジェクトへの参照が与えられる

70099483621100
70099483621100
"fuga"
"hoge"
70099483599120
70099483621100
"メソッドの引数"
70099478495120
70099478495120
70099478458140


70099478458140

## 破壊的メソッド

* 自分自身の内容を変更するメソッド
    * 他の変数の参照先にも影響する
        * メソッド名に「!」をつけるのが慣習(例外もある)

In [144]:
v1 = "foo1"
v2 = v1     ### v2はv1と同じ文字列を参照する
p v1        #=> "foo1"
p v2        #=> "foo1" v2はv1と同じ文字列を参照
p v1.chop   #=> "foo"
p v1        #=> "foo1" chopの後でも参照先は変更されない
p v2        #=> "foo1"
p v1.chop!  #=> "foo"
p v1        #=> "foo"  chop!の後はv1の参照先が変更されている
p v2        #=> "foo"  chop!の後はv2の参照先まで変更されている！！

"foo1"
"foo1"
"foo"
"foo1"
"foo1"
"foo"
"foo"
"foo"


"foo"

## 文字列とシンボルの違い

* 文字列
    * 変数に代入すると、たとえ同じ値であってもそれぞれ異なるオブジェクトを指す
* シンボル
    * 変数が同じオブジェクトを指す

In [145]:
v1="foo"
v2="foo"
v3="foo"
p v1.object_id   #=> (69836757139180など)  ※ すべて異なるオブジェクトID
p v2.object_id   #=> (69836757139160など)
p v3.object_id   #=> (69836757139140など)
v1=:foo
v2=:foo
v3=:foo
p v1.object_id   #=> (2466908など)  ※ すべて同じオブジェクトID
p v2.object_id   #=> (2466908など)
p v3.object_id   #=> (2466908など)


70099470490480
70099470490460
70099470490440
3255708
3255708
3255708


3255708

# 配列

* Arrayクラスのインスタンス
    * 配列リテラル
        * 要素を角カッコで囲み、要素の間をカンマで区切ることで生成する
    * Arrayクラスのインスタンスを生成することで配列を作成可能
    * 初期値は第二引数やブロックで指定
    * 第二引数で指定したオブジェクトは同一オブジェクト
    * 破壊的メソッドで変更するとすべての配列の値が変更されることになる
    * ブロックで初期化することですべての配列の値が変更されることを防ぐことができる

In [150]:
p "配列リテラル : 要素を角カッコで囲み、要素の間をカンマで区切ることで生成する"
p a = [10, true, "foo", :bar]   #=> [10, true, "foo", :bar]
p a[0]                          #=> 10

p "初期値は第二引数やブロックで指定"
p v1 = Array.new(3)               #=> [nil, nil, nil]
p v1.length                       #=> 3
p v2 = Array.new(3,"foo")         #=> ["foo", "foo", "foo"]
p v3 = Array.new(3){"bar"}        #=> ["bar", "bar", "bar"]
p Array.new(3){|v| v = "hoge"}    #=> ["hoge", "hoge", "hoge"]
p Array.new(3){|v| v}             #=> [0, 1, 2]
p Array.new(3){|v| v + 1}         #=> [1, 2, 3]
p Array.new(3){|v| v + 3}         #=> [3, 4, 5]

p "第二引数で指定したオブジェクトは同一オブジェクト"
p v4 = Array.new(3,"fuga")        #=> ["fuga", "fuga", "fuga"]
p v4[0].object_id                 #=> (69924207752260など)
p v4[1].object_id                 #=> (69924207752260など上と同じ)
p v4[2].object_id                 #=> (69924207752260など上と同じ)

p "破壊的メソッドで変更するとすべての配列の値が変更されることになる"
p v5 = Array.new(3,"haga")        #=> ["haga", "haga", "haga"]
v5[0].replace("zzz")
p v5                              #=> ["zzz", "zzz", "zzz"]

p "ブロックで初期化することですべての配列の値が変更されることを防ぐことができる"
p v6 = Array.new(3){"xxx"}          #=> ["xxx", "xxx", "xxx"]
p v6[0].object_id                 #=> (70099484225320など)
p v6[1].object_id                 #=> (70099484225300など上と異なる)
p v6[2].object_id                 #=> (70099484225280など上と異なる)
v6[0].replace("yyy")
p v6                                #=> ["yyy", "xxx", "xxx"]

"配列リテラル : 要素を角カッコで囲み、要素の間をカンマで区切ることで生成する"
[10, true, "foo", :bar]
10
"初期値は第二引数やブロックで指定"
[nil, nil, nil]
3
["foo", "foo", "foo"]
["bar", "bar", "bar"]
["hoge", "hoge", "hoge"]
[0, 1, 2]
[1, 2, 3]
[3, 4, 5]
"第二引数で指定したオブジェクトは同一オブジェクト"
["fuga", "fuga", "fuga"]
70099470305680
70099470305680
70099470305680
"破壊的メソッドで変更するとすべての配列の値が変更されることになる"
["haga", "haga", "haga"]
["zzz", "zzz", "zzz"]
"ブロックで初期化することですべての配列の値が変更されることを防ぐことができる"
["xxx", "xxx", "xxx"]
70099484225320
70099484225300
70099484225280
["yyy", "xxx", "xxx"]


["yyy", "xxx", "xxx"]

* 二次元配列
* パーセント記法による配列の生成
    * %Wでダブルクオート、%wでシングルクオート区切りの配列
    * 素間は空白区切り
* サイズを超えた要素への代入
    * 配列のサイズは要素の代入によって動的に変わる
    * 上限を超えた場合はnilを返す
* 添え字演算子
    * 配列の要素への参照や代入
        * 構文ではなくArrayクラスのメソッドとして実装されている
        * 参照
            * []メソッド
        * 代入
            * []=メソッド
    * 添え字として負の整数や複数の整数を指定できる
    * <font color="Red">要素を指定して代入</font>

In [153]:
p "二次元配列"
p b = [[1,2],["a","b"]]         #=> [[1, 2], ["a", "b"]]
p b[0]                          #=> [1,2]
p b[0][1]                       #=> 2

p "パーセント記法による配列の生成"
foo="foo"
bar="bar"
p %W(abc def #{foo})              #=> ["abc", "def", "foo"]
p %w(ghi jkl #{bar})              #=> ["ghi", "jkl", "\#{bar}"]
p %W(abc def #{foo}).join("&")    #=> "abc&def&foo"

p "サイズを超えた要素への代入"
p v1 = ["hoge"]                  #=> ["hoge"]
v1[3] = "fuga"
p v1                             #=> ["hoge", nil, nil, "fuga"]
p v1[9]                          #=> nil

p "添え字演算子"
p v1 = ["a","b","c","d","e"]      #=> ["a", "b", "c", "d", "e"]
p v1[0]                           #=> "a"   ### 先頭の要素
p v1[-1]                          #=> "e"   ### 末尾の要素
p v1[4-1]                         #=> "d"   ### 4-1 = 3  先頭を0番目として3の要素
p v1[-4]                          #=> "b"   ### 末尾を1番目として4番目の要素
p v1[-9]                          #=> nil   ### 上限を超えたらnil
p v1[2,3]                         #=> ["c", "d", "e"]  ### 先頭を0番目としてインデックス2(2番目の要素)からそれ自身を含めて後ろ3つの要素を表示
p v1[3,2]                         #=> ["d", "e"]  ### 先頭を0番目としてインデックス3(3番目の要素)からそれ自身を含めて後ろ2つの要素を表示

p "要素を指定して代入"
p v2 = ["a","b","c","d","e"]      #=> ["a", "b", "c", "d", "e"] ### 先頭を0番目の要素として1,2番目の要素b,cが操作対象
v2[1,2] = "f"                     ### インデックス1を含めて2要素分を1要素として"f"を代入。
p v2                              #=> ["a", "f", "d", "e"]  ### 1,2番目の要素だったb,cが1つの要素fになった
v2[1,2] = ["g","h"]               ### インデックス1を含めて1要素目にg,2要素目にhを代入。
p v2                              #=> ["a", "g", "h", "e"]  ### 1,2番目の要素f,dにg,hが代入されて置き換わった

"二次元配列"
[[1, 2], ["a", "b"]]
[1, 2]
2
"パーセント記法による配列の生成"
["abc", "def", "foo"]
["ghi", "jkl", "\#{bar}"]
"abc&def&foo"
"サイズを超えた要素への代入"
["hoge"]
["hoge", nil, nil, "fuga"]
nil
"添え字演算子"
["a", "b", "c", "d", "e"]
"a"
"e"
"d"
"b"
nil
["c", "d", "e"]
["d", "e"]
"要素を指定して代入"
["a", "b", "c", "d", "e"]
["a", "f", "d", "e"]
["a", "g", "h", "e"]


["a", "g", "h", "e"]

# メソッド(関数)

* オブジェクトに何らかの処理を行わせる場合はメソッドとして定義しておく
    * クラスの中で定義する
* メソッドの定義
* メソッドの実行
* 引数はデフォルト値を設定できる
* キーワード引数 (Ruby2.0以降)
    * 仮引数名とデフォルト値をコロンで結びつけて定義できる
        * 呼び出しにハッシュオブジェクトを渡すことで呼び出し側でどの引数にどのような値を渡したかを明示できる
* キーワード引数に任意の引数を使う
* 一番最後に評価した値が返る
* returnで明示的に返り値を指定できる

In [1]:
### メソッドaddの定義
def add(a,b)  # def式で指定する引数(ここでは a, b) は仮引数と呼ぶ
  a + b
end

#セミコロンを使うと1行で書ける
def add2(a,b); a + b; end

p "メソッドの実行"
p add(1,2)   # メソッドを実行する際に記述する引数(ここでは1, 2)を実引数と呼ぶ
p add2(3,4)

p "仮引数はデフォルト値を設定できる"
def add3(a, b=100)
  a + b
end

p add3(1,2)   # 3
p add3(1)     # 101

p "キーワード引数 (Ruby2.0以降)"
def add4(a:, b: 100)
  a + b
end

p add4(a: 1, b: 2)  # 3
p add4(a: 1)        # 101
#p add4(b: 1)        # ArgumentError

p "キーワード引数に任意の引数を使う"
def add5(a:, b: 100, **z)
  p z    # コンソール上には {:c=>3, :d=>4} が表示される
  a + b
end

p add5(a: 1, b: 2, c: 3, d: 4)   # 3, コンソール上には {:c=>3, :d=>4} が表示される

p "一番最後に評価した値が返る"
def add6(a, b)
  a + b
  a + b + b
end

p add6(3,4)   # 11

p "returnで明示的に返り値を指定できる"
def add7(a, b)
  return a + b
  a + b + b
end

p add7(3,4)   # 7

"\u30E1\u30BD\u30C3\u30C9\u306E\u5B9F\u884C"
3
7
"\u4EEE\u5F15\u6570\u306F\u30C7\u30D5\u30A9\u30EB\u30C8\u5024\u3092\u8A2D\u5B9A\u3067\u304D\u308B"
3
101
"\u30AD\u30FC\u30EF\u30FC\u30C9\u5F15\u6570 (Ruby2.0\u4EE5\u964D)"
3
101
"\u30AD\u30FC\u30EF\u30FC\u30C9\u5F15\u6570\u306B\u4EFB\u610F\u306E\u5F15\u6570\u3092\u4F7F\u3046"
{:c=>3, :d=>4}
3
"\u4E00\u756A\u6700\u5F8C\u306B\u8A55\u4FA1\u3057\u305F\u5024\u304C\u8FD4\u308B"
11
"return\u3067\u660E\u793A\u7684\u306B\u8FD4\u308A\u5024\u3092\u6307\u5B9A\u3067\u304D\u308B"
7


7

# クラス

* トップレベル
    * クラスの定義式やモジュールの定義式など、定義式の外は「トップレベル」と呼ばれる
    * トップレベルで定義されたメソッドはグローバルなサブルーチンのように使用できる
        * メソッド呼び出しの際レシーバを記述しない
        * いわゆる関数のように、どこからでもグローバルに呼び出す事ができる(一部の例外を除く)
* クラスの定義
    * メソッドの定義
* クラスのインスタンスの生成
* クラスのメソッドを実行

In [168]:
### BasicFooクラスの定義
class BasicFoo
  ### testメソッドの定義
  def test
    1
  end
end

### BasicFooクラスのインスタンスの生成
foo = BasicFoo.new

### BasicFooクラスのtestメソッドを実行
p foo.test

1


1

## attr_* について

例を追って說明。

* https://www.xmisao.com/2014/02/10/ruby-attr-accessor-attr-reader-attr-writer.html
* http://bryankawa.hatenablog.com/entry/2017/01/28/150537

* 以下Personクラスがあったとする
    * nameメソッドが定義されてないのでエラーになる

In [8]:
class Person
end

person = Person.new
person.name

NoMethodError: undefined method `name' for #<Person:0x007f210f1a4e60>

* nameのreaderメソッドを定義する
    * nameメソッドを読み込めた
    * ただ、名前が割り当てられた訳ではない

In [1]:
class Person
  def name ### readerメソッド
    @name  ### 単純にインスタンス変数を返す
  end  
end

person = Person.new
p person.name
person.name = "Jim"

nil


NoMethodError: undefined method `name=' for #<Person:0x007f8eca78ad10>

* nameのwriterメソッドを定義する 
    * reader, writerメソッドを使う事でインスタンス変数@nameを定義して呼び出すことができるようになった

In [13]:
class Person
  def name=(str)  ### writerメソッド
    @name = str   ### インスタンス変数に値を設定 
  end
  
  def name
    @name
  end  
end

person = Person.new
p person.name
person.name = "Jim"

nil


"Jim"

* 毎回reader,writerメソッドを書くのが面倒
  * attr_reader, attr_writerを使うと簡単に書ける
      * attr_reader : ゲッターを定義する
      * attr_writer : セッターを定義する

In [7]:
class Person
  attr_reader :name
  attr_writer :name
end

person = Person.new
p person.name
person.name = "Jim"

nil


"Jim"

* attr_accessorを使うともっと簡単に書ける
    * セッターとゲッターを共に定義する
* `attr_accessor :name` について
    * 「`:name`」はインスタンス変数「`@name`」のこと<font color="red">ではない</font>
    * セッター、ゲッターの<font color="red">メソッドの名前</font>が「name, name=」になるということを表している

In [10]:
class Person
  attr_accessor :name
end

person = Person.new
p person.name
person.name = "Jim"

nil


"Jim"

* インスタンス変数 `@name` はpersonオブジェクトに設定することができる
  * 他のメソッドからも呼び出せる

In [12]:
class Person
  attr_accessor :name
  
  def greeting
    "Hello #{@name}"
  end
end

person = Person.new
person.name = "Jim"
person.greeting

"Hello Jim"

### attr_accessorの挙動について

* http://qiita.com/jordi/items/7baeb83788c7a8f2070d
* attr_accessor
    * 引数はシンボルで指定
    * シンボルで指定された名前の2つのメソッドを定義する
        * メソッド1
            * シンボルで指定された名前のメソッドを定義する
            * そのメソッドはシンボルの名前の先頭に 「`@`」 を付加したインスタンス変数の値を取り出す
            * attr_readerのこと
        * メソッド2
            * シンボルで指定されたメソッド名の後ろに 「=」 を付加した名前のメソッドを定義する
            * そのメソッドは、引数を一つ取り、シンボルの名前の先頭に 「`@`」 を付加したインスタンス変数に設定する
            * attr_writerのこと

## ゲッター、セッターを設定せずにインスタンス変数を取り出す

* ゲッターが設定されていないとインスタンス変数にアクセスできない

In [2]:
class Fuga
  # attr_reader :messages    ### ここがコメントアウトされてなければエラーにならない
  def initialize
    @message = "Hello"
  end
end

puts Fuga.new.message  ### attr_readerにて設定されているべきmessageというメソッドが無いのでエラーになる

NoMethodError: undefined method `message' for #<Fuga:0x007f5a29daa950 @message="Hello">

* `instance_variables` でインスタンス変数を取得できる
* `instance_variable_get` でインスタンス変数の中身を取得できる

In [3]:
class Fuga
  def initialize
    @message = "Hello"
  end
end

puts Fuga.new.instance_variables
puts Fuga.new.instance_variable_get(:@message)

[:@message]
Hello
